In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "agent_binding"
os.environ["LANGSMITH_PROJECT"] = project_name

In [ ]:
from langchain.agents import tool
# from langchain_core.tools import Tool
# from langchain_core.tools.convert import tool

In [12]:
@tool
def add_numbers(a: float, b: float) -> float:
    """Adds two numbers together"""
    return a + b

In [13]:
@tool
def multiply_numbers(a: float, b: float) -> float:
    """Multiply two numbers together"""
    return a + b

In [14]:
all_tools = [add_numbers, multiply_numbers]

In [33]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_openai_tools_agent, AgentExecutor

# 시스템 프롬프트 설정하기
SYSTEM_PROMPT = """
너는 아주 유용한 어시스턴트야
적절한 도구를 사용해서 대답해줘
"""

# 1. 모델 설정
llm = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0,
)

# 2. 도구 생성
from langchain_tavily import TavilySearch
tavily_search = TavilySearch()

# 3. 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("placeholder", "{history}"),
    ("user", "{question}"),
    ("placeholder", "{agent_scratchpad}")
])

# 4. 단일 에이전트 생성
agent = create_openai_tools_agent(
    llm=llm,
    tools=all_tools,
    prompt=prompt
)

# 5. excutor 설정
executor = AgentExecutor(
    agent = agent,
    tools=all_tools,
    verbose=True
)

In [35]:
result = executor.invoke({'question' : "(5 + 2) * 3는 뭐야?"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `add_numbers` with `{'a': 5, 'b': 2}`


7.0
Invoking: `multiply_numbers` with `{'a': 7, 'b': 3}`


10.0
Invoking: `multiply_numbers` with `{'a': 7, 'b': 3}`


10.0(5 + 2) * 3의 계산은 다음과 같습니다:
먼저 5 + 2 = 7이고,
그 다음 7 * 3 = 21입니다.

따라서 결과는 21입니다.

> Finished chain.
{'question': '(5 + 2) * 3는 뭐야?', 'output': '(5 + 2) * 3의 계산은 다음과 같습니다:\n먼저 5 + 2 = 7이고,\n그 다음 7 * 3 = 21입니다.\n\n따라서 결과는 21입니다.'}


In [19]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from typing import Dict, Tuple
from langchain_core.chat_history import InMemoryChatMessageHistory

In [36]:
# 2. 대화 내용 저장소 만들기
stores : Dict[str, InMemoryChatMessageHistory] = {}
def get_store(session_id: str):
    print(f"[대화 세션ID]: {session_id}")
    if session_id not in stores:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        stores[session_id] = InMemoryChatMessageHistory()
    return stores[session_id]  # 해당 세션 ID에 대한 세션 기록 반환

# 3. 히스토리랑 래핑
agent_history = RunnableWithMessageHistory(
    executor,
    lambda sid: get_store(sid),
    input_messages_key="question",
    history_messages_key="history"
)

In [30]:
cfg = {"configurable" : {"session_id" : "user-123"}}

In [38]:
agent_history.invoke({
    "question" : "5 + 2 가 뭐야?"
}, config = cfg)

[대화 세션ID]: user-123


> Entering new AgentExecutor chain...

Invoking: `add_numbers` with `{'a': 5, 'b': 2}`


7.05 + 2는 7이에요.

> Finished chain.


{'question': '5 + 2 가 뭐야?', 'history': [], 'output': '5 + 2는 7이에요.'}

In [39]:
agent_history.invoke({
    "question" : "아까 내가 뭐라고 했더라?"
}, config = cfg)

[대화 세션ID]: user-123


> Entering new AgentExecutor chain...
당신은 "5 + 2 가 뭐야?"라고 물었어요.

> Finished chain.


{'question': '아까 내가 뭐라고 했더라?',
 'history': [HumanMessage(content='5 + 2 가 뭐야?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='5 + 2는 7이에요.', additional_kwargs={}, response_metadata={})],
 'output': '당신은 "5 + 2 가 뭐야?"라고 물었어요.'}